In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [2]:
spark = (
    SparkSession
    .builder
    .appName("landing_to_bronze")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "24G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7fad444f-e27f-497a-8fe6-ce8b7bae9183;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 635ms :: artifacts dl 7ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from ce

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *

In [5]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes/'
silver_path_restricted = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes_restrito/'
BRONZE_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'


In [6]:
def genereteSymlinkManifest(spark:SparkSession, bucket:str) -> None:
    """
    Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
    """
    delta_table = DeltaTable.forPath(spark, bucket)
    delta_table.generate("symlink_format_manifest")

In [7]:
def pathExists(path, spark:SparkSession):
    sc = spark.sparkContext
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
        sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
        sc._jsc.hadoopConfiguration(),
    )
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))

In [8]:
def lastDate(delta_path:str, spark:SparkSession):
    if not pathExists(delta_path + 'job_control/last_date', spark):
        try:
            df_temp = (
                spark
                .read
                .format("delta")
                .load(delta_path)
            )

            list_result = (
                df_temp
                .agg({"timestamp_kafka": "max"}).collect()
            )

            last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        except:
            last_date = ('01-01-2021 12:00AM')

        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    else:
        df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
        last_date = df.collect()[0].asDict()['last_date']
    
    return parser.parse(last_date)

In [9]:
def updateLastDate(df:DataFrame, spark:SparkSession):
    list_result = (
        df
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
    df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
    df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    
    return last_date

In [11]:
last_date = lastDate(silver_path, spark)

In [10]:
last_date = parser.parse('21-03-2021 08:00AM')

In [11]:
df_table = (
    spark
    .read
    .format("delta")
    .load(BRONZE_PATH_PRODUTO_CORES)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp_kafka")>last_date)
    .limit(10)
)

In [13]:
df_table.toPandas()['DESC_COR_PRODUTO']

0        AZUL
1    VERMELHO
2        ROXO
3     AMARELO
4        ROSA
5       VERDE
6     LARANJA
7       VERDE
8       VERDE
9        AZUL
Name: DESC_COR_PRODUTO, dtype: object

In [17]:
df_table.toPandas().columns

Index(['offset', 'timestamp_kafka', 'year', 'month', 'day', 'hour',
       'CODIGO_CLIENTE', 'NOME_CLIENTE', 'FILIAL', 'CPF_CNPJ', 'PF_PJ',
       'PONTUALIDADE', 'CODIGO_CONTATO', 'TIPO_VAREJO', 'ENDERECO', 'RG_IE',
       'CEP', 'TIPO_LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'PAIS',
       'COMPLEMENTO', 'DDDTELEFONE', 'TELEFONE', 'ANIVERSARIO', 'OBSERVACAO',
       'SEXO', 'TIPO_TELEFONE', 'EMAIL', 'ULTIMA_COMPRA', 'ESTADO_CIVIL',
       'PROFISSAO', 'DDD_CELULAR', 'CELULAR', 'LIMITE_CREDITO',
       'DATA_CADASTRAMENTO', 'VENDEDOR', 'ID_CLIENTE', 'DATA_ATUALIZACAO',
       'PRIMEIRA_COMPRA', 'QTDE_COMPRA', 'MAIOR_COMPRA', 'QTDE_DEVOLUCAO',
       'VALOR_TOTAL_COMPRA', 'VALOR_TOTAL_DEVOLUCAO', 'CADASTRO_INVALIDO',
       'FILIAL_CADASTRO', 'AJUSTADO_MKT', 'DATA_CRM', 'ID_CLIENTE_ZENDESK',
       'DATA_HORA_ATUALIZACAO_ZENDESK', 'AJUSTES', 'DATA_HORA_CRITICA_ZENDESK',
       'FIDELIDADE_PUC', 'FIDELIDADE_HERING', 'STATUS_ZENDESK',
       'DESC_ERRO_ZENDESK', 'EMAIL_VALIDO', '

In [16]:
df_table_restricted = (
    df_table
    .select(f.col('CPF_CNPJ'), f.col('CODIGO_CLIENTE'), f.col('NOME_CLIENTE'), 
            f.col('ENDERECO'), f.col('RG_IE'), f.col('CEP'), f.col('NUMERO'), 
            f.col('BAIRRO'), f.col('TELEFONE'), f.col('EMAIL'), 
            f.col('CELULAR'), f.col('ID_CLIENTE'), f.col('__op'))
    .withColumn("CPF_CNPJ_HASH", f.sha2(f.col('CPF_CNPJ'), 256))
    .withColumn("CODIGO_CLIENTE_HASH", f.sha2(f.col('CODIGO_CLIENTE'), 256))
    .withColumn("NOME_CLIENTE_HASH", f.sha2(f.col('NOME_CLIENTE'), 256))
    .withColumn("ENDERECO_HASH", f.sha2(f.col('ENDERECO'), 256))
    .withColumn("RG_IE_HASH", f.sha2(f.col('RG_IE'), 256))
    .withColumn("CEP_HASH", f.sha2(f.col('CEP'), 256))
    .withColumn("NUMERO_HASH", f.sha2(f.col('NUMERO'), 256))
    .withColumn("BAIRRO_HASH", f.sha2(f.col('BAIRRO'), 256))
    .withColumn("TELEFONE_HASH", f.sha2(f.col('TELEFONE'), 256))
    .withColumn("EMAIL_HASH", f.sha2(f.col('EMAIL'), 256))
    .withColumn("CELULAR_HASH", f.sha2(f.col('CELULAR'), 256))
    .withColumn("ID_CLIENTE_HASH", f.sha2(f.col('ID_CLIENTE'), 256))
)

In [17]:
df_table_hash = (
    df_table
    .withColumn("CPF_CNPJ", f.sha2(f.col('CPF_CNPJ'), 256))
    .withColumn("CODIGO_CLIENTE", f.sha2(f.col('CODIGO_CLIENTE'), 256))
    .withColumn("NOME_CLIENTE", f.sha2(f.col('NOME_CLIENTE'), 256))
    .withColumn("ENDERECO", f.sha2(f.col('ENDERECO'), 256))
    .withColumn("RG_IE", f.sha2(f.col('RG_IE'), 256))
    .withColumn("CEP", f.sha2(f.col('CEP'), 256))
    .withColumn("NUMERO", f.sha2(f.col('NUMERO'), 256))
    .withColumn("BAIRRO", f.sha2(f.col('BAIRRO'), 256))
    .withColumn("TELEFONE", f.sha2(f.col('TELEFONE'), 256))
    .withColumn("EMAIL", f.sha2(f.col('EMAIL'), 256))
    .withColumn("CELULAR", f.sha2(f.col('CELULAR'), 256))
    .withColumn("ID_CLIENTE", f.sha2(f.col('ID_CLIENTE'), 256))
)

In [20]:
# Snapshot hash
if df_table_hash.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_r = df_table_hash.where(f.col('__op') == 'r')
    (
        df_table_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path + "checkpoint")
        .save(silver_path)
    )
    
# Insert and Update
if df_table_hash.where(f.col('__op') != 'r').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    df_table_u = df_table_hash.where(f.col('__op') != 'r')
    df_table_u_unique = df_table_u.orderBy('offset', ascending = False).dropDuplicates(["CPF_CNPJ"])
    (
        silver_table.alias('silver')
        .merge(df_table_u_unique.alias('updates'),'silver.CPF_CNPJ = updates.CPF_CNPJ')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table_hash.where(f.col('__op') == 'd').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.delete(col('__op') == 'd')
    

In [19]:
# Snapshot restrited
if df_table_restricted.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_restricted_r = df_table_restricted.where(f.col('__op') == 'r')
    (
        df_table_restricted_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path_restricted + "checkpoint")
        .save(silver_path_restricted)
    )
    
# Insert and Update
if df_table_restricted.where(f.col('__op') != 'r').count() > 0:
    silver_table_restricted = DeltaTable.forPath(spark, silver_path_restricted)
    df_table_restricted_u = df_table_restricted.where(f.col('__op') != 'r')
    df_table_restricted_u_unique = df_table_restricted_u.orderBy('offset', ascending = False).dropDuplicates(["CPF_CNPJ"])
    (
        silver_table_restricted.alias('silver')
        .merge(df_table_restricted_u_unique.alias('updates'),'silver.CPF_CNPJ = updates.CPF_CNPJ')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table_restricted.where(f.col('__op') == 'd').count() > 0:
    silver_table_restricted = DeltaTable.forPath(spark, silver_path_restricted)
    silver_table_restricted.delete(f.col('__op') == 'd')
    

In [ ]:
if df_table.count() > 0:
    updated_date = updateLastDate(df_table, spark)
    genereteSymlinkManifest(spark, silver_path)
    print(updated_date)

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
df_silver_hash_t = (
    spark
    .read
    .format("delta")
    .load(silver_path)
)

df_silver_restricted_t = (
    spark
    .read
    .format("delta")
    .load(silver_path_restricted)
)

In [ ]:
df_silver_hash_t.join(df_silver_restricted_t,df_silver_hash_t.PEDIDO == df_silver_restricted_t.PEDIDO,"inner").limit(100).toPandas()

In [ ]:
df_pd = df_silver_teste.toPandas()

In [ ]:
df_pd

In [ ]:
df_pd[df_pd["__op"] != "r"]